# Cube conversion

This notebooks creates an optimized version of each `SEG-Y` cube.
The exact format (`HDF5`, `BLOSC` or their quantized versions) depends on `FORMAT` and `QUANTIZE` parameters.

Pseudocode of this notebook looks like:

```python
for each cube:
    mkdir
    infer geometry
    if SHOW, log to std.out
    
    convert segy to a desired format
```

* The parameter `paths` controls which cubes are converted
* `RECREATE` determines whether already converted volumes are re-converted
* `FORMAT` and `QUANTIZE` determine the exact format to convert to
* `SHOW` allows to control whether results are shown in the notebook itself
* `DRY` can be used to check which operations will happen, without actually executing them

In [ ]:
import os
import sys
import warnings
from tqdm.auto import tqdm
from glob import glob
import matplotlib.pyplot as plt

sys.path.append('..')
from seismiqb import SeismicGeometry, plot_image

In [ ]:
paths = sorted(glob('/data/seismic_data/seismic_interpretation/CUBE_*/amplitudes*.s*y'))
[print(path) for path in paths]

RECREATE = True
FORMAT = 'blosc'
QUANTIZE = True
SHOW = True

DRY_RUN = False

In [ ]:
%%time
for path_cube in tqdm(paths, ncols=800):
    if not os.path.exists(path_cube):
        continue
    
    path_converted = '.'.join((os.path.splitext(path_cube)[0],
                               ('q' if QUANTIZE else '') + FORMAT))
    if os.path.exists(path_converted) and not RECREATE:
        print(f'{path_converted} already exists, skipping')
        continue
    
    if DRY_RUN:
        print(f'Will convert ::: {path_cube}\nto           ::: {path_converted}\n')
        continue
    
    if SHOW:
        print('▆'*60); print('▆'*60);
        print('Working with', path_cube)

    geometry = SeismicGeometry(
        path_cube,
        headers=SeismicGeometry.HEADERS_POST_FULL,
        index_headers = SeismicGeometry.INDEX_POST,
        collect_stats=True, spatial=True, recollect=True
    )
    qmap = geometry.quality_map

    if SHOW:
        geometry.print()
        geometry.print_textual()

        geometry.show()
        geometry.show_quality_map()
        plt.show()
    
    geometry_converted = geometry.convert(format=FORMAT, quantize=QUANTIZE)
    
    if SHOW:
        geometry_converted.print()
        print('\n'*3)